In [1]:
import pandas as pd
import numpy as np

In [30]:
test = pd.read_csv("/Users/philippheitmann/Downloads/streamlit_prediction_database_prediciton_2021_09_02.csv")

In [31]:
test

,Unnamed: 0,Unnamed: 0.1,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
0,0,0,0.24,0.11,0.13,0.17,0.08,0.07,0.20
1,1,2021-09-01,0.10,0.25,0.18,0.12,0.21,0.06,0.07
2,2021-09-02,NaN,0.11,0.25,0.19,0.11,0.20,0.07,0.07


In [5]:
test = test.iloc[1]["Unnamed: 0.1"]

'2021-09-01'

In [6]:
test

,Unnamed: 0,Unnamed: 0.1,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
0,0,0,0.24,0.11,0.13,0.17,0.08,0.07,0.20
1,1,2021-09-01,0.10,0.25,0.18,0.12,0.21,0.06,0.07
2,2021-09-02,NaN,0.11,0.25,0.19,0.11,0.20,0.07,0.07


In [7]:
def merge_date(df_prediction_2):

    prediction_date = get_date_n_days_ago().replace("_","-")

    #df_prediction_2.index = [prediction_date]
    df_prediction_2["Unnamed: 0"] = prediction_date

    return df_prediction_2

In [9]:
test.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], inplace = True)

In [11]:
prediction = test

In [18]:
yes = prediction.iloc[[1]]
yes.index = ["2021-09-01"]
yes

,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
2021-09-01,0.1,0.25,0.18,0.12,0.21,0.06,0.07


In [19]:
today = prediction.iloc[[2]]
today.index = ["2021-09-02"]
today

,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
2021-09-02,0.11,0.25,0.19,0.11,0.2,0.07,0.07


In [25]:
today.index.name = "Date"

In [26]:
today

,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
Date,,,,,,,
2021-09-02,0.11,0.25,0.19,0.11,0.2,0.07,0.07


In [27]:
yes.to_csv("/Users/philippheitmann/Downloads/yesterday.csv")

In [28]:
today.to_csv("/Users/philippheitmann/Downloads/today.csv")

In [29]:
pd.read_csv("/Users/philippheitmann/Downloads/yesterday.csv", index_col="Date" )

,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
Date,,,,,,,
2021-09-01,0.1,0.25,0.18,0.12,0.21,0.06,0.07


In [55]:
test = pd.read_csv("/Users/philippheitmann/Downloads/streamlit_prediction_database_prediciton_2021_09_01 (4).csv", index_col="Unnamed: 0")
test

,AFD,CDU,GRUENE,FDP,SPD,LINKE,OTHER
0,0.24,0.11,0.13,0.17,0.08,0.07,0.20
2021-09-01,0.10,0.25,0.18,0.12,0.21,0.06,0.07


In [59]:
test["AFD"]*100

0             24.0
2021-09-01    10.0
Name: AFD, dtype: float64

In [53]:
len(["black" for i in range(8)] + ["white" for i in range(8)])


16

In [50]:
len([
    "gold",
    "gold",
    "gold",
    "gold",
    "gold",
    "gold",
    "gold",
    "gold",
    "black",
    "black",
    "black",
    "black",
    "black",
    "black",
    "black",
    "black",
])

16

In [2]:
def most_popular_tweets(df, tweet_date):
    ''' Find and return the most liked/ most retweeted positive and negative tweets overall and per party
    --> returns two df most_liked_tweets, most_retweeted_tweets '''

    #Create empty dataframe for most likes and most retweeted tweets
    most_liked_tweets = pd.DataFrame()
    most_retweeted_tweets = pd.DataFrame()

    # Create lists for iterations for party and sentiment
    parties = ['CDU', 'SPD', 'GRUENE', 'FDP', 'LINKE', 'AFD', 'OTHER']
    sentiments = [-2, 2]  #-2: negative, +2: positive

    #Columns to keep
    columns = [
        'username', 'text', 'tweet_date', 'retweet_count', 'reply_count', 'like_count',
        'sentiment', "party"
    ]
    #'username'
    
    # Clean dataset columns:
    df["retweet_count"] = df["retweet_count"].astype(int)
    df["reply_count"] = df["reply_count"].astype(int)
    df["like_count"] = df["like_count"].astype(int)
    df["sentiment"] = df["sentiment"].astype(int)
    
    #Only keep relevant columns of Twitter dataframe
    df = df[columns]

    #Change to datemtime
    df["tweet_date"] = pd.to_datetime(df["tweet_date"])

    #Change format of datetime to 2021-08-25
    df["tweet_date"] = df["tweet_date"].dt.strftime('%Y-%m-%d')

    # Filter df for day of tweet
    df = df[df["tweet_date"] == tweet_date]

    #Get most liked/ retweeted Tweets over all parties
    for sentiment in sentiments:

        #Filter for sentiment
        tweets_tmp = df[df["sentiment"] == sentiment].reset_index()

        # Find tweet with most likes and concatenate to final df
        most_liked_tweet = tweets_tmp.iloc[[tweets_tmp["like_count"].idxmax()]]
        most_liked_tweets = pd.concat([most_liked_tweets, most_liked_tweet],
                                      ignore_index=True)
        most_liked_tweets["party"] = "OVERALL"

        # Find tweet with most retweets and concatenate to final df
        most_retweeted_tweet = tweets_tmp.iloc[[
            tweets_tmp["retweet_count"].idxmax()
        ]]
        most_retweeted_tweets = pd.concat(
            [most_retweeted_tweets, most_retweeted_tweet], ignore_index=True)
        most_retweeted_tweets["party"] = "OVERALL"

    #Get most liked/ retweeted Tweets for each party
    for party in parties:
        
        #if party == "OTHER":
        #    import ipdb
        #    ipdb.set_trace()

        tweets_party_df = df[df["party"] == party]

        #Find tweets with most likes & most retweets per party and concatenate them to the final dfs
        for sentiment in sentiments:
            
            
            #Filter for sentiment
            tweets_tmp = pd.DataFrame(tweets_party_df[
                tweets_party_df["sentiment"] == sentiment].reset_index())
            

            
            # Find tweet with most likes and concatenate to final df
            most_liked_tweet = tweets_tmp.iloc[[
                tweets_tmp["like_count"].idxmax()
            ]]
            most_liked_tweets = pd.concat(
                [most_liked_tweets, most_liked_tweet], ignore_index=True)

            # Find tweet with most retweets and concatenate to final df
            most_retweeted_tweet = tweets_tmp.iloc[[
                tweets_tmp["retweet_count"].idxmax()
            ]]
            most_retweeted_tweets = pd.concat(
                [most_retweeted_tweets, most_retweeted_tweet],
                ignore_index=True)

    return most_liked_tweets, most_retweeted_tweets


In [8]:
def load_and_clean_csv(df):
    '''
    Function loads DF data from the Twitter API+Sentiment and returns a cleaned DF
    '''
    #import ipdb
    #ipdb.set_trace()
    # rename columns
    df = df.rename(columns={"tweet_created_at": "tweet_date",
                                "public_metrics.retweet_count": "retweet_count",
                                "public_metrics.reply_count": "reply_count",
                                "public_metrics.like_count": "like_count",
                                "profile_created_at": "profile_creation_date",
                                "public_metrics.followers_count": "followers_count",
                                "public_metrics.following_count": "following_count",
                                "public_metrics.tweet_count": "user_tweet_count"
                                })

    # Including only columns that we want to use in the future
    df = df[['party',
                'tweet_date',
                'author_id',
                'tweet_id',
                'text',
                'source',
                'retweet_count',
                'reply_count',
                'like_count',
                'profile_creation_date',
                'followers_count',
                'following_count',
                'user_tweet_count',
                'location',
                'sentiment',
                'username'
                ]]

    # Clean dataset columns:
    # Change dtype
    df["tweet_date"] = df["tweet_date"].astype(str)
    #df = df[df.tweet_date.str.match('(\d{4}-\d{2}-\d{2}.\d{2}:\d{2}:\d{2})')]
    #df = df[(df.tweet_date.str.len() == 23) | (df.tweet_date.str.len() == 24)]
    df['tweet_date'] = df['tweet_date'].str.slice(0,19)
    df["tweet_date"] = pd.to_datetime(df["tweet_date"])

    df["party"] = df["party"].astype(str)
    df["profile_creation_date"] = df["profile_creation_date"].astype(str)
    df = df[df.profile_creation_date.str.match(
        '(\d{4}-\d{2}-\d{2}.\d{2}:\d{2}:\d{2})')]
    df['profile_creation_date'] = df['profile_creation_date'].str.slice(0,19)
    df["profile_creation_date"] = pd.to_datetime(df["profile_creation_date"])
    # Drop duplicates
    #df = df.drop_duplicates()
    # Transform sentiment to numeric type
    dict_to_numeric = {"negative": -2, "neutral": 1, "positive": 2}
    df["sentiment"].replace(dict_to_numeric, inplace=True)

    return df

In [9]:
tweet_date = "2021-08-31"

In [10]:
data1 = pd.read_csv("/Users/philippheitmann/Downloads/tweet_database_08_31.csv" , lineterminator="\n")

In [11]:
#from project_delphi.features import load_and_clean_csv

In [62]:
data1.shape[0]

276129

In [76]:
tdata = pd.DataFrame({"no_tweets_today": data1.shape[0], "no_tweets_total":1_000_000}, index=[0])

In [79]:
tdata.to_csv("../../raw_data/no_of_tweets.csv", index=False)

In [89]:
pd.read_csv("../../raw_data/no_of_tweets.csv")

,no_tweets_today,no_tweets_total
0,276129,1000000


In [12]:
data = load_and_clean_csv(data1)

ParserError: Unknown string format: OTHER

In [ ]:
data.party.unique()

In [75]:
pd.read_csv("../../raw_data/test_tweets.csv")

/Users/philippheitmann/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,party,tweet_date,author_id,tweet_id,text,source,retweet_count,reply_count,like_count,profile_creation_date,followers_count,following_count,user_tweet_count,location,sentiment,username
0,SPD,2021-08-31 23:59:24,229106593,1432855573284937730,Heftige Merkel-Attacke - ist Olaf Scholz eine ...,Twitter Web App,1.0,0.0,0.0,2010-12-21 14:55:20,56.0,118.0,7921.0,Cuxhaven,1.0,13und13
1,SPD,2021-08-31 23:58:48,3625766896,1432855420482334725,@larsklingbeil Klingbeil und was ist mit der S...,Twitter for iPhone,0.0,0.0,0.0,2015-09-12 00:29:18,3.0,174.0,326.0,Wilder Süden,1.0,RitschF
2,SPD,2021-08-31 23:58:42,1170234519951159296,1432855397963083780,"Hauptsächlich Esken das ref, spielt sich vor d...",Twitter Web App,0.0,0.0,1.0,2019-09-07 07:17:17,1680.0,1434.0,43503.0,irgendwo bei Lummerland,1.0,dummchens
3,SPD,2021-08-31 23:52:45,1170234519951159296,1432853899514421248,"Scholz ist blöder als lang..\nMister Teflon.,\...",Twitter Web App,1.0,1.0,1.0,2019-09-07 07:17:17,1680.0,1434.0,43503.0,irgendwo bei Lummerland,1.0,dummchens
4,SPD,2021-08-31 23:58:12,1730034720,1432855271869661184,"""Menschen dürfen nicht GEGENEINANDER ausgespie...",Twitter Web App,0.0,0.0,0.0,2013-09-04 22:24:43,7.0,46.0,5377.0,NaN,1.0,HmmelinkReinhol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422182,CDU,2021-08-27 00:01:20,18309034,1431044121393836034,@abendblatt Herr Röhse hat jahrelang den ÖPNV...,Twitter Web App,0.0,0.0,0,2008-12-22 15:42:26,309,1505.0,14250.0,Planet Erde,-2.0,kickahh
422183,CDU,2021-08-27 00:03:59,49985382,1431044787705090048,Auch bei #Illner gab es also #KeineUnion #Wege...,Twitter for Android,1.0,0.0,6,2009-06-23 14:13:04,1619,563.0,26867.0,Ruhrgebiet,1.0,4therecords
422184,CDU,2021-08-27 00:03:08,1381013960917409800,1431044573481086982,"@akk @cducsubt Man sollte sich langsam Fragen,...",Twitter Web App,0.0,0.0,0,2021-04-10 22:40:34,2,15.0,494.0,NaN,1.0,1K4tMqqn0KLzpVw
422185,CDU,2021-08-27 00:03:08,502657567,1431044571606179845,"Foto: Armin Laschet, über dts Nachrichtenagent...",Blog2Social APP,0.0,0.0,0,2012-02-25 07:51:40,544,101.0,59881.0,Günzburg,1.0,BSAktuell


In [72]:
pd.read_csv("../../raw_data/test_tweets.csv").drop(columns=["Unnamed: 0"]).to_csv("../../raw_data/test_tweets.csv", index=False)

/Users/philippheitmann/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.dtypes

In [ ]:
tweets_tmp = data[(data["sentiment"]==-2) & (data["party"]=="OTHER")].reset_index()
tweets_tmp.iloc[[tweets_tmp["like_count"].idxmax()]]

In [ ]:
data

In [ ]:
most_popular_tweets(data, tweet_date)

In [ ]:
data1.party.unique()